# Continual Pretraining With TinyLlama-120M

추가 사전학습 실습용 코드입니다. 자세한 설명은 다음 글을 참고하길 바랍니다: [글 링크](https://jkspace.notion.site/11-20-238b1a84a5a9409fb895ce349fef85e1?pvs=4)

본 실습에서는

1. 한국어 뉴스 데이터로 사전학습을 하고
2. 상담 데이터로 **추가 사전학습(Continual Pretraining)**을 해도
3. 첫 번째 도메인(뉴스 데이터)에 대한 정보를 모델이 잘 기억하고 있는지 확인하는 것을 목표로 합니다.

📌 특히 Colab에서 돌릴 때, HuggingFace (이하, HF) Hub에 데이터셋, 체크포인트를 백업하는 것을 권장합니다.


# Set Environment

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate

In [ ]:
# HF login
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
import os
import torch
from transformers import (
    TrainingArguments, Trainer, DataCollatorForLanguageModeling,
    AutoTokenizer, LlamaForCausalLM, LlamaConfig
)
import datasets as ds

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    n_gpus = torch.cuda.device_count()
else:
    device = torch.device("cpu")

# Tokenizer

토크나이저는 이미 한국어로 학습된 것을 사용합니다.
- 토크나이저 HF repo: [beomi/llama-2-ko-7b](https://huggingface.co/beomi/llama-2-ko-7b)

In [ ]:
tokenizer_id = "beomi/llama-2-ko-7b" # 학습 데이터에 사용한 토크나이저

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_id,
    # add_bos_token=True,
    # add_eos_token=True # If not mentioned, Llama Tokenizer doesn't automatically add eos_token
)
tokenizer.pad_token = tokenizer.unk_token

tokenizer

LlamaTokenizerFast(name_or_path='beomi/llama-2-ko-7b', vocab_size=46336, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46331: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46332: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46333: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46334: A

# Generating Function

프롬프트와 추론 함수를 미리 정의해 놓습니다.

In [ ]:
from tqdm import tqdm

prompts = [
    ### first pretrain
    # "KT AI 스피커",
    # "미국 증시가 내리자",
    # "마이크로소프트는 올 들어 현재까지 주가가 23% 이상 하락했다",
    "이번 제휴 협약은 K POP 콘서트",
    # "금융정보를 받은 통신사는 인공지능 AI 을 통해",

    ### second pretrain
    "불안감과 스트레스로",

    # "최근에 무기력감이 심해지고, 에너지가 없어서 아무것도 하기 싫은",
    # "스트레스 받아서 삶이 너무 힘들어요",
]

def infer(prompts:list, model, tokenizer, device=torch.device("cpu"), print_flag: bool=True):
    model.to(device)

    results = []
    for idx, prompt in tqdm(enumerate(prompts), desc="Generating", total=len(prompts)):
        model_input = tokenizer(
            prompt,
            return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                    **model_input,
                    max_new_tokens=100,
                    do_sample=True,
                    top_k=50,
                    top_p=0.95,
                    num_return_sequences=2,
                    )
            output_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            results.append(output_texts)

    if print_flag:
        for r in results:
            print("--- START ---")
            for idx, rr in enumerate(r):
                print(f"--- {idx} ---")
                print(rr)

    return results

# Create Dataset

사전학습용 데이터셋을 만듭니다.
HF DataSet 또는 jsonl 파일에서 데이터셋을 만들 수 있습니다.

이미 만들어져 있는 데이터셋를 사용해도 됩니다.

- 첫 번째 사전학습 데이터셋 HF repo: [lectura/naver_news_1024](https://huggingface.co/datasets/lectura/naver_news_1024)
- 두 번째 사전학습 데이터셋 HF repo: [lectura/counsel-ko_1024](https://huggingface.co/datasets/lectura/counsel-ko_1024)

## Create Text Dataset from HF DataSet

In [ ]:
import datasets as ds

base_data_path = "daekeun-ml/naver-news-summarization-ko"
text_column = "document" # 원본 데이터셋에서 사용할 열 이름

dataset = ds.load_dataset(base_data_path)

dataset = ds.concatenate_datasets(
    [dataset[k] for k in dataset.keys()] # train, valid, test 모두 학습에 사용
    )

text_dataset = dataset.map(
    lambda x: {"text": x[text_column]},
    batched=True,
    remove_columns=dataset.column_names
)

print(text_dataset)
print(text_dataset['text'][0])

Dataset({
    features: ['text'],
    num_rows: 27400
})
앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.


## Create Text Dataset from JSONL File

jsonl 파일은 다음 형식의 데이터를 저장하고 있어야 합니다.

```
{"text": "첫 번째 문장"}
{"text": "두 번째 문장"}
{"text": "세 번째 문장"}
```

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import datasets as ds

base_data_path = "counsel_kor_data.jsonl"
text_column = "text"

with jsonlines.open(base_data_path) as f:
    data = [l for l in f]

data_dict = {
    'text': [d[text_column] for d in data]
}

text_dataset = ds.Dataset.from_dict(data_dict)

print(text_dataset)
print(text_dataset['text'][0])

Dataset({
    features: ['text'],
    num_rows: 13234
})
저 사실, 약간 중2병 같은 걸 증상을 보이고 있습니다.
그래서 대학교를 가면서도 그런 부분들이 조금은 개선될 거라 생각했는데, 이번 학기 들어서 오히려 더 심해졌더라구요.
혼자 있는 공간에서 자주 가사를 쓰거나 소설이나 시를 쓰는 편이고요. 그것들을 쓰면서 살아가는 것 같은 기분이 들어서 그것들이 점점 늘어나고 있는 것 같습니다.
그리고 그런 것들이 나쁜 것이 아니라고 느껴지기도 하는데, 이상하게도 내가 나쁜 걸 하고 있다는 기분도 듭니다. 머리속에서 아무 생각도 나지 않을 때도 가사를 쓰거나 소설을 써본 적이 있는데, 그게 그렇게 마음이 편안해지는 느낌이라서 계속 써왔습니다.
그런데 이런 것들을 과도하게 하면 나중에 문제가 생길까봐 걱정이 되기도 합니다. 저랑 같은 고민을 하시는 분들이 있는지, 이런 경우에는 어떤 조치를 취해야할까요?


## Concatenate All Text and Split into Chunk Size

각 문장을 토큰화 한 후 하나로 붙이고, `chunk_size` 크기만큼 잘라서 새로운 데이터셋을 만듭니다.

In [ ]:
chunk_size = 1024

def split_chunk(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i+chunk_size]

def tokenize(batch):
    return tokenizer(
        batch["text"],
    )

tokenized_dataset = text_dataset.map(
    tokenize,
    batched=True,
    remove_columns = text_dataset.column_names
)

input_ids = []
for ids in tokenized_dataset['input_ids']:
    # if tokenizer don't automatically add eos_token
    ids.append(tokenizer.eos_token_id)
    input_ids.extend(ids)

input_batch = []
for chunk in list(split_chunk(input_ids, chunk_size=chunk_size)):
    if len(chunk) == chunk_size:
        input_batch.append(chunk)

temp = {"input_ids": input_batch}

train_dataset = ds.Dataset.from_dict(temp)

print(train_dataset)
print(train_dataset[0])

Map:   0%|          | 0/13234 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 1034
})
{'input_ids': [1, 32060, 32591, 29892, 35746, 32030, 29906, 45007, 32280, 32287, 32288, 32541, 35560, 32777, 29889, 13, 38325, 41972, 31517, 32015, 35316, 32348, 34051, 32039, 32936, 31354, 33494, 45041, 44315, 44061, 29892, 32263, 32365, 30827, 37923, 33905, 32071, 32254, 31435, 45064, 35802, 29889, 13, 40569, 32123, 34221, 32012, 34609, 32015, 32387, 32185, 32706, 36910, 32144, 32026, 31517, 34260, 32354, 32244, 31527, 29889, 38108, 32308, 32185, 32103, 43656, 32033, 32280, 38969, 37923, 32004, 33868, 35483, 35499, 31137, 32123, 32033, 35145, 29889, 13, 36176, 32348, 34859, 33841, 32666, 35167, 35453, 30811, 32657, 33353, 29892, 32309, 32108, 31136, 32916, 33841, 32287, 32281, 33194, 35406, 31136, 35806, 29889, 33797, 43357, 32497, 39040, 32019, 30811, 34091, 38466, 32015, 32387, 32185, 32706, 36910, 31286, 32801, 44898, 36768, 32975, 29892, 33144, 32601, 35996, 37702, 40781, 33575, 36314, 32542, 32801, 43954, 29889, 13, 40

In [ ]:
# disk에 저장
data_path = "naver_news_1024"
train_dataset.save_to_disk(data_path)

# HF에 업로드
# train_dataset.push_to_hub("naver_news_1024", private=True)

Saving the dataset (0/1 shards):   0%|          | 0/12945 [00:00<?, ? examples/s]

# First Pretraining

먼저 한국어 뉴스 데이터로 사전학습을 합니다. \
colab에서는 10000 steps가 ~60 시간 걸린다고 뜹니다. V100 GPU 8개로는 ~2시간 걸렸습니다.

이미 학습된 모델을 사용해도 됩니다.

- 첫 번째 사전학습 모델 HF repo: [lectura/TinyLlama-120M-news](https://huggingface.co/lectura/TinyLlama-120M-news/settings)

In [ ]:
### TODO 1. check we're using the right data, model, tokenizer
data_path = "lectura/naver_news_1024"
context_length = 1024 # 학습 데이터 chunk size와 동일하게 설정함. Llama2는 4096.
base_model_id = "nickypro/tinyllama-110M" # Config만 가져올 것

### if data_path is on local disk
# dataset = ds.load_from_disk(data_path)

### if data_path is huggingface_hub repo
dataset = ds.load_dataset(data_path, split="train")

print(f"dataset: {dataset}")
print(f"len(dataset[0]['input_ids']): {len(dataset[0]['input_ids'])}")
print(f"dataset[0]['input_ids'][:10]: {dataset[0]['input_ids'][:10]}")


dataset: Dataset({
    features: ['input_ids'],
    num_rows: 12945
})
len(dataset[0]['input_ids']): 1024
dataset[0]['input_ids'][:10]: [1, 37407, 45100, 33191, 32619, 38080, 32172, 32412, 30708, 32464]


In [ ]:
vocab_size = len(tokenizer)

config = LlamaConfig.from_pretrained(
    base_model_id,
    max_position_embeddings = context_length,
    vocab_size = vocab_size,
)

model = LlamaForCausalLM(config)

config

LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "max_position_embeddings": 1024,
  "model_type": "llama",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_key_value_heads": 12,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 46336
}

In [ ]:
print("Model parameter size:",
      sum(p.numel() for p in model.parameters() if p.requires_grad)
      )


Model parameter size: 120539904


In [ ]:
# 사전학습 전 결과
_ = infer(prompts, model, tokenizer, device=device)

In [ ]:
dir_name = "TinyLlama-120M-news"

args = TrainingArguments(
        output_dir=dir_name,
        save_steps=500,
        save_total_limit=2,
        overwrite_output_dir=True,

    ### Hub에 올리는 설정
        # push_to_hub=True,
        # hub_model_id=dir_name,
        # hub_strategy="all_checkpoints", # 모든 체크포인트를 각각 폴더에 저장
        # hub_private_repo=True,
    ### END
        logging_steps=1,

        fp16=True,
        gradient_checkpointing=True,

        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=16, # global batch 128
        max_steps = 10000,
        learning_rate=4e-4, # default 0.00005
    )

model.train()

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=dataset
)

trainer.train()
trainer.save_model()

바로 추가 사전학습을 진행하려면 GPU 메모리를 비우기 위해 런타임을 다시 시작하는 것을 권장한다.

# Continual Pretraining

상담 데이터로 추가 사전학습을 진행해 봅시다.

⭐ 학습률을 충분히 작게 설정하면 기존의 정보를 잊는 것을 방지할 수 있습니다.


* 추가 사전학습한 모델 HF repo: [lectura/TinyLlama-120M-news-counsel-lr1e-5](https://huggingface.co/lectura/TinyLlama-120M-news-counsel-lr1e-5)

In [ ]:
data_path = "lectura/counsel-ko_1024" # 추가 사전학습 에 사용할 데이터셋
context_length = 1024 # 학습 데이터 chunk size와 동일하게 설정
base_model_id = "lectura/TinyLlama-120M-news" # 추가 사전학습할 모델

In [ ]:
### if data_path is on local disk
# dataset = ds.load_from_disk(data_path)

### if data_path is huggingface_hub repo
dataset = ds.load_dataset(data_path, split="train")

print(f"dataset: {dataset}")
print(f"len(dataset[0]['input_ids']): {len(dataset[0]['input_ids'])}")
print(f"dataset[0]['input_ids'][:10]: {dataset[0]['input_ids'][:10]}")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

dataset: Dataset({
    features: ['input_ids'],
    num_rows: 1034
})
len(dataset[0]['input_ids']): 1024
dataset[0]['input_ids'][:10]: [1, 32060, 32591, 29892, 35746, 32030, 29906, 45007, 32280, 32287]


In [ ]:
model = LlamaForCausalLM.from_pretrained(base_model_id)

In [ ]:
# 추가 사전학습 전 결과
first_pretrain_results = infer(prompts, model, tokenizer, device=device)

첫 번째 사전학습에 사용된 하이퍼파라미터에서 `learning_rate, num_train_epochs`만 변경했습니다.\
colab에서 `20` epoch을 돌리는데 ~1시간이 걸립니다.\
`Trainig Loss = 4.5` 정도 되면 괜찮은 결과를 볼 수 있습니다.

In [ ]:
### TODO 3. Go to configs.py and check TrainingArguments
dir_name = "TinyLlama-120M-news-counsel"

args = TrainingArguments(
        output_dir=dir_name,
        save_strategy="epoch",
        save_total_limit=1,
        overwrite_output_dir=True,

    ### Hub에 올리는 설정
        # push_to_hub=True,
        # hub_model_id=dir_name,
        # hub_strategy="all_checkpoints", # 모든 체크포인트를 각각 폴더에 저장
        # hub_private_repo=True,
    ### END
        logging_steps=1,

        fp16=True,
        gradient_checkpointing=True,

        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=16, # global batch 128
        num_train_epochs=20,
        learning_rate=1e-5, # Should be less than first pretraining
    )

model.train()

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=dataset
)

trainer.train()
trainer.save_model()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,12.783700
2,12.459500
3,11.912100
4,11.662800
5,11.432700
6,11.148800
7,10.920900
8,10.844800
9,10.531000
10,10.376700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# 추가 사전학습 후 결과
second_pretrain_results = infer(prompts, model, tokenizer, device=device)

Generating:   0%|          | 0/8 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Generating: 100%|██████████| 8/8 [00:19<00:00,  2.44s/it]

--- START ---
--- 0 ---
KT AI 스피커 중간관리 IT AI 반도체 기업 알뜰폰사에 배터리 민감단 가 필요한 데이터센터에 데이터를 돌아다니며 관련 기술을 개발하고 협력하는 방안을 병행한다. 7일 업계에 따르면 김근숙 의원과 통신벨트를 소명을 비롯한 관계자들이 인식이 아닌 이상 성과에 답하고 있다. 하나T의 3나노 공정의 파일의 파운드리 점유율은 16%로 AI 반도체와 AI 관련 기술을 융합해 인력을 높이겠다는 목표를 가진
--- 1 ---
KT AI 스피커 서비스가 끝 조용히 아반객이라고 생각하는데요. 뛰는 시간입니다. 무료고 싶지 않습니다. 지금은 최근에 전공에 살고 싶지 않아서 걱정이라 도와주면서 그만도 그 사람들이 매일 신경 안 가는 걸까요. 그런데 이 문제를 어떻게 해결해야 할까요.
--- START ---
--- 0 ---
미국 증시가 내리자다. 하토민로의테오 시우들의 모습 사진 전날 현지시간 미국 뉴욕증권거래소에서 10%대 고배를 마친 뉴욕 농심켈이 혼조했던 곳인가제들이 시설 투자를 펼치며 하루가 멀다 하고 계열사들에게 연락을 많이 했는데 있다. 이들은 이날 나스닥시에 2.13%씩 추락해 1021년 펀드운용 기준 심사를 통과하고 현재까지 공격적으로 경기침체에도 불구하고 바닥을 다지는 것일
--- 1 ---
미국 증시가 내리자 2020년 7월 14일 현지시간 미국 뉴욕상업거래소에서 미주지역땐 수입이 720억 달러 미국서 19억 달러 미달러 감소했다. 이에 따라 2300선 아래로 떨어질 예정이다. 로이터뉴스114 데이비즈 영국 런던빌딩투니가 지난 5월 10일 독립기념일을 맞아 구스모빌과 화폐의 비전을 발표하며 ‘탑건 매’
--- START ---
--- 0 ---
마이크로소프트는 올 들어 현재까지 주가가 23% 이상 하락했다.
--- 1 ---
마이크로소프트는 올 들어 현재까지 주가가 23% 이상 하락했다. 그러나 시가총액에선 아직 바닥을 잡기 힘들다면 저의 발언을 하지 않은 시점이지만 지금은 제 역할을 하지 못하고 있다. 몇 년 동

추가 사전학습을 진행한 모델이 뉴스 데이터에 관한 프롬프트에는 뉴스 데이터 내용을 생성하고, 상담 데이터에 관한 프롬프트는 상담 데이터 내용을 생성하는 것을 볼 수 있을 것입니다.

In [ ]:
# 결과 비교
print("*** First pretrain results *** ")
print(first_pretrain_results[0][0])
print("-"*10)
print(first_pretrain_results[-1][0])

print("*** Second pretrain results *** ")
print(second_pretrain_results[0][0])
print("-"*10)
print(second_pretrain_results[-1][0])

*** First pretrain results *** 
KT AI 스피커 와 DB “�� Str 자연스럽게 선보일 예정길 우리들은 이데일리 김수정 기자 한국항공우주연구원 나항공 지연에 힘입어 제주항공이 항공우주체계 조종에 속도를 내고 있다. 항공우주 공급청은 여객선 여객접송 항공 우주선 8기 열선을 마치고 5일 발사장 이송을 마치며 내 우주로 향하는 목적인 우주 단지로 온더CC를 비행했다. 골프장 운임이 가능한 위성재생원 승객도 다음달 10일까지 우주
----------
스트레스 받아서 삶이 너무 힘들어요. 다시 조용히 옮기는 기업가치 제고 의미가...통계 공동 경쟁력 강화시너지 강연승 하나은행 회장이 최근에 취임했습니다. 취임 후 첫 행보로 한꺼번에 쏟아졌던 명동에 오르면서 신사업 분위기를 느낀 기업들을 본격적으로 희생했고 결국 무분별하게 손을 잡아줬습니다. 4일 하나은행이 계열사 빅데이터에 “이 총수가 본인의 찾아온 것인지 아니면 무분별한 생활일 것이다”는 표현을 하고 있습니다. 하나은행 관계자는 “하나은행에서 수많은
*** Second pretrain results *** 
KT AI 스피커 중간관리 IT AI 반도체 기업 알뜰폰사에 배터리 민감단 가 필요한 데이터센터에 데이터를 돌아다니며 관련 기술을 개발하고 협력하는 방안을 병행한다. 7일 업계에 따르면 김근숙 의원과 통신벨트를 소명을 비롯한 관계자들이 인식이 아닌 이상 성과에 답하고 있다. 하나T의 3나노 공정의 파일의 파운드리 점유율은 16%로 AI 반도체와 AI 관련 기술을 융합해 인력을 높이겠다는 목표를 가진
----------
스트레스 받아서 삶이 너무 힘들어요. 가끔 스트레스도 떨어져요. 또 또 급하다보니 스트레스를 더 어렵지 않을까 저에게 마음이 더 불분명함도 있어서면 정말 바쁘고요. 지금은 지금은 정말 사나고 스트레스를 하게 될 것 같아요. 그래서 그렇게 너무 많아서 이럴 것이란 부담이 됩니다. 그래서 그냥 제가 이럴 때 제가 너무 힘들어요. 제가 이럴 때는 이제 생각해봅니다. 제가 하